<a href="https://colab.research.google.com/github/rockey1006/Variations-on-Variational-Autoencoders/blob/master/Copy_of_VaDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import

In [0]:
!git clone https://github.com/slim1017/VaDE.git
!pip install --upgrade scikit-learn==0.17.1
!pip install --upgrade keras==1.1.0
!pip install --upgrade theano==1.0.0

Cloning into 'VaDE'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.
    100% |████████████████████████████████| 7.9MB 5.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/17/68/59/e7417b53bb95a8bbbf17f8ab90fa1b8e985141e76de1d110b7
Successfully built scikit-learn
yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.17.1 which is incompatible.
mlxtend 0.14.0 has requirement scikit-learn>=0.18, but you'll have scikit-learn 0.17.1 which is incompatible.
imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.17.1 which is incompatible.
fancyimpute 0.4.2 has requirement scikit-learn>=0.19.1, but you'll have scikit-learn 0.17.1 which is incompatible.
  Found existing installation: scikit-learn 0.20.2
    Uninstalling scikit-learn-0.20.2:
      Successfully uninstalled scikit-learn-0.20.2
    100% |████████████████████████████████

In [0]:
# overwrite training.py from the VADE 
!yes | cp -rf VaDE/training.py  ../usr/local/lib/python3.6/dist-packages/keras/engine/

In [0]:
from keras import backend
print(backend._BACKEND)
from keras import backend as K
import importlib;
import os
def set_keras_backend(backend):
    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        importlib.reload(K)
        assert K.backend() == backend
print ("Change Keras Backend to Theano")        
set_keras_backend("theano")  
from keras import backend; print(backend._BACKEND)

import pickle

Using TensorFlow backend.


tensorflow
Change Keras Backend to Theano


Using Theano backend.


theano


In [0]:
import numpy as np
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
import scipy.io as scio
import gzip
from six.moves import cPickle
import sys
import  theano
import  theano.tensor as T
import math
from sklearn import mixture
from sklearn.cluster import KMeans
from keras.models import model_from_json

import warnings
warnings.filterwarnings("ignore")


In [0]:
import requests
import tarfile

# Model Setup

In [0]:
def download_cifar100():
    url = 'https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz'
    
    local_filename = url.split('/')[-1]

    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    
    tar = tarfile.open(local_filename, "r:gz")
    tar.extractall()
    tar.close()

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [0]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)
    
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.)
    return z_mean + K.exp(z_log_var / 2) * epsilon

def cluster_acc(Y_pred, Y):
  from sklearn.utils.linear_assignment_ import linear_assignment
  assert Y_pred.size == Y.size
  D = max(Y_pred.max(), Y.max())+1
  w = np.zeros((D,D), dtype=np.int64)
  for i in range(Y_pred.size):
    w[Y_pred[i], Y[i]] += 1
  ind = linear_assignment(w.max() - w)
  return sum([w[i,j] for i,j in ind])*1.0/Y_pred.size, w

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def displayImage(image):
    displayList=np.array(image).T
    im1 = Image.fromarray(displayList)
    im1.show()

In [0]:
def load_data(dataset):

    path = 'VaDE/dataset/'+dataset+'/'
    if dataset == 'mnist':
        path = path + 'mnist.pkl.gz'
        if path.endswith(".gz"):
            f = gzip.open(path, 'rb')
        else:
            f = open(path, 'rb')
    
        if sys.version_info < (3,):
            (x_train, y_train), (x_test, y_test) = cPickle.load(f)
        else:
            (x_train, y_train), (x_test, y_test) = cPickle.load(f, encoding="bytes")
    
        f.close()
        x_train = x_train.astype('float32') / 255.
        x_test = x_test.astype('float32') / 255.
        x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
        x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
        X = np.concatenate((x_train,x_test))
        Y = np.concatenate((y_train,y_test))
        
    if dataset == 'reuters10k':
        data=scio.loadmat(path+'reuters10k.mat')
        X = data['X']
        Y = data['Y'].squeeze()
        
    if dataset == 'har':
        data=scio.loadmat(path+'HAR.mat')
        X=data['X']
        X=X.astype('float32')
        Y=data['Y']-1
        X=X[:10200]
        Y=Y[:10200]
        
    if dataset == 'cifer100':
        try:
            train_data = unpickle('cifar-100-python/train')
            test_data = unpickle('cifar-100-python/test')
        except:
            download_cifar100()
            train_data = unpickle('cifar-100-python/train')
            test_data = unpickle('cifar-100-python/test')
        X_train = train_data[b'data'].astype('float32') / 255.
        Y_train = train_data[b'fine_labels']
        X_test = test_data[b'data'].astype('float32') / 255.
        Y_test = test_data[b'fine_labels']
        X = np.concatenate((X_train, X_test))
        Y = np.concatenate((Y_train, Y_test))

    return X,Y

def config_init(dataset):
    if dataset == 'mnist':
        return 784,3000,10,0.002,0.002,10,0.9,0.9,1,'sigmoid'
    if dataset == 'reuters10k':
        return 2000,15,4,0.002,0.002,5,0.5,0.5,1,'linear'
    if dataset == 'har':
        return 561,120,6,0.002,0.00002,10,0.9,0.9,5,'linear'
    
    # original_dim,epoch,n_centroid,lr_nn,lr_gmm,decay_n,decay_nn,decay_gmm,alpha,datatype
    if dataset == 'cifer100':
        return 3072,3,100,0.002,0.002,5,0.5,0.5,1,'linear'
    

        
def gmmpara_init():
    
    theta_init=np.ones(n_centroid)/n_centroid
    u_init=np.zeros((latent_dim,n_centroid))
    lambda_init=np.ones((latent_dim,n_centroid))
    
    theta_p=theano.shared(np.asarray(theta_init,dtype=theano.config.floatX),name="pi")
    u_p=theano.shared(np.asarray(u_init,dtype=theano.config.floatX),name="u")
    lambda_p=theano.shared(np.asarray(lambda_init,dtype=theano.config.floatX),name="lambda")
    return theta_p,u_p,lambda_p

In [0]:
def get_gamma(tempz):
    temp_Z=T.transpose(K.repeat(tempz,n_centroid),[0,2,1])
    temp_u_tensor3=T.repeat(u_p.dimshuffle('x',0,1),batch_size,axis=0)
    temp_lambda_tensor3=T.repeat(lambda_p.dimshuffle('x',0,1),batch_size,axis=0)
    temp_theta_tensor3=theta_p.dimshuffle('x','x',0)*T.ones((batch_size,latent_dim,n_centroid))
    
    temp_p_c_z=K.exp(K.sum((K.log(temp_theta_tensor3)-0.5*K.log(2*math.pi*temp_lambda_tensor3)-\
                       K.square(temp_Z-temp_u_tensor3)/(2*temp_lambda_tensor3)),axis=1))+1e-10
    return temp_p_c_z/K.sum(temp_p_c_z,axis=-1,keepdims=True)

def vae_loss(x, x_decoded_mean):
    Z=T.transpose(K.repeat(z,n_centroid),[0,2,1])
    z_mean_t=T.transpose(K.repeat(z_mean,n_centroid),[0,2,1])
    z_log_var_t=T.transpose(K.repeat(z_log_var,n_centroid),[0,2,1])
    u_tensor3=T.repeat(u_p.dimshuffle('x',0,1),batch_size,axis=0)
    lambda_tensor3=T.repeat(lambda_p.dimshuffle('x',0,1),batch_size,axis=0)
    theta_tensor3=theta_p.dimshuffle('x','x',0)*T.ones((batch_size,latent_dim,n_centroid))
    
    p_c_z=K.exp(K.sum((K.log(theta_tensor3)-0.5*K.log(2*math.pi*lambda_tensor3)-\
                       K.square(Z-u_tensor3)/(2*lambda_tensor3)),axis=1))+1e-10

    gamma=p_c_z/K.sum(p_c_z,axis=-1,keepdims=True)
    gamma_t=K.repeat(gamma,latent_dim)
    
    if datatype == 'sigmoid':
        loss=alpha*original_dim * objectives.binary_crossentropy(x, x_decoded_mean)\
        +K.sum(0.5*gamma_t*(latent_dim*K.log(math.pi*2)+K.log(lambda_tensor3)\
                            +K.exp(z_log_var_t)/lambda_tensor3\
                            +K.square(z_mean_t-u_tensor3)/lambda_tensor3),axis=(1,2))\
        -0.5*K.sum(z_log_var+1,axis=-1)\
        -K.sum(K.log(K.repeat_elements(theta_p.dimshuffle('x',0),batch_size,0))*gamma,axis=-1)\
        +K.sum(K.log(gamma)*gamma,axis=-1)
    else:
        loss=alpha*original_dim * objectives.mean_squared_error(x, x_decoded_mean)\
        +K.sum(0.5*gamma_t*(latent_dim*K.log(math.pi*2)+K.log(lambda_tensor3)\
                            +K.exp(z_log_var_t)/lambda_tensor3\
                            +K.square(z_mean_t-u_tensor3)/lambda_tensor3),axis=(1,2))\
        -0.5*K.sum(z_log_var+1,axis=-1)\
        -K.sum(K.log(K.repeat_elements(theta_p.dimshuffle('x',0),batch_size,0))*gamma,axis=-1)\
        +K.sum(K.log(gamma)*gamma,axis=-1)
        
    return loss


def load_pretrain_weights(vade,dataset):
    ae = model_from_json(open('VaDE/pretrain_weights/ae_'+dataset+'.json').read())
    ae.load_weights('VaDE/pretrain_weights/ae_'+dataset+'_weights.h5')
    vade.layers[1].set_weights(ae.layers[0].get_weights())
    vade.layers[2].set_weights(ae.layers[1].get_weights())
    vade.layers[3].set_weights(ae.layers[2].get_weights())
    vade.layers[4].set_weights(ae.layers[3].get_weights())
    vade.layers[-1].set_weights(ae.layers[-1].get_weights())
    vade.layers[-2].set_weights(ae.layers[-2].get_weights())
    vade.layers[-3].set_weights(ae.layers[-3].get_weights())
    vade.layers[-4].set_weights(ae.layers[-4].get_weights())
    sample = sample_output.predict(X,batch_size=batch_size)
    if dataset == 'mnist':
        g = mixture.GMM(n_components=n_centroid,covariance_type='diag')
        g.fit(sample)
        u_p.set_value(floatX(g.means_.T))
        lambda_p.set_value((floatX(g.covars_.T)))
    if dataset == 'reuters10k':
        k = KMeans(n_clusters=n_centroid)
        k.fit(sample)
        u_p.set_value(floatX(k.cluster_centers_.T))
    if dataset == 'har':
        g = mixture.GMM(n_components=n_centroid,covariance_type='diag',random_state=3)
        g.fit(sample)
        u_p.set_value(floatX(g.means_.T))
        lambda_p.set_value((floatX(g.covars_.T)))
    print ('pretrain weights loaded!')
    return vade

def lr_decay():
    if dataset == 'mnist':
        adam_nn.lr.set_value(floatX(max(adam_nn.lr.get_value()*decay_nn,0.0002)))
        adam_gmm.lr.set_value(floatX(max(adam_gmm.lr.get_value()*decay_gmm,0.0002)))
    else:
        adam_nn.lr.set_value(floatX(adam_nn.lr.get_value()*decay_nn))
        adam_gmm.lr.set_value(floatX(adam_gmm.lr.get_value()*decay_gmm))
    print ('lr_nn:%f'%adam_nn.lr.get_value())
    print ('lr_gmm:%f'%adam_gmm.lr.get_value())
    
def epochBegin(epoch):

    if epoch % decay_n == 0 and epoch!=0:
        lr_decay()
    '''
    sample = sample_output.predict(X,batch_size=batch_size)
    g = mixture.GMM(n_components=n_centroid,covariance_type='diag')
    g.fit(sample)
    p=g.predict(sample)
    acc_g=cluster_acc(p,Y)
    
    if epoch <1 and ispretrain == False:
        u_p.set_value(floatX(g.means_.T))
        print ('no pretrain,random init!')
    '''
    gamma = gamma_output.predict(X,batch_size=batch_size)
    acc=cluster_acc(np.argmax(gamma,axis=1),Y)
    global accuracy
    accuracy+=[acc[0]]
    if epoch>0 :
        #print ('acc_gmm_on_z:%0.8f'%acc_g[0])
        print ('acc_p_c_z:%0.8f'%acc[0])
    if epoch==1 and dataset == 'har' and acc[0]<0.77:
        print ('=========== HAR dataset:bad init!Please run again! ============')
        sys.exit(0)
        
class EpochBegin(Callback):
    def on_epoch_begin(self, epoch, logs={}):
        epochBegin(epoch)

# Run

In [0]:
dataset = 'cifer100'
db = sys.argv[1]
if db in ['mnist','reuters10k','har', 'cifer100']:
    dataset = db
print ('training on: ' + dataset)
ispretrain = False
batch_size = 100
latent_dim = 10
intermediate_dim = [500,500,2000]
theano.config.floatX='float32'
accuracy=[]
X,Y = load_data(dataset)

training on: cifer100


In [0]:
# Model Parameters
epoch = 6
n_centroid = 100  # number of categories
lr_nn = 0.005
lr_gmm = 0.005
decay_n = 5
decay_nn = 0.5
decay_gmm = 0.5
alpha = 1
datatype = 'linear'
original_dim = X[0].size

theta_p,u_p,lambda_p = gmmpara_init()

In [0]:
x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim[0], activation='relu')(x)
h = Dense(intermediate_dim[1], activation='relu')(h)
h = Dense(intermediate_dim[2], activation='relu')(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
h_decoded = Dense(intermediate_dim[-1], activation='relu')(z)
h_decoded = Dense(intermediate_dim[-2], activation='relu')(h_decoded)
h_decoded = Dense(intermediate_dim[-3], activation='relu')(h_decoded)
x_decoded_mean = Dense(original_dim, activation=datatype)(h_decoded)

In [0]:
Gamma = Lambda(get_gamma, output_shape=(n_centroid,))(z)
sample_output = Model(x, z_mean)
gamma_output = Model(x,Gamma)

In [0]:
vade = Model(x, x_decoded_mean)

if ispretrain == True:
    vade = load_pretrain_weights(vade,dataset)

adam_nn= Adam(lr=lr_nn,epsilon=1e-4)
adam_gmm= Adam(lr=lr_gmm,epsilon=1e-4)

vade.compile(optimizer=adam_nn, loss=vae_loss, 
             add_optimizer=adam_gmm,
             add_trainable_weights=[theta_p,u_p,lambda_p])

epoch_begin=EpochBegin()

In [0]:
vade.fit(X, X,
        shuffle=True,
        nb_epoch=epoch,
        batch_size=batch_size,
        callbacks=[epoch_begin])

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Epoch 1/6
60000/60000 [==============================] - 107s - loss: 337816.9282   
acc_p_c_z:0.01000000
Epoch 2/6
60000/60000 [==============================] - 431s - loss: 211.1764   
acc_p_c_z:0.01000000
Epoch 3/6
60000/60000 [==============================] - 507s - loss: 204.8619   
acc_p_c_z:0.01000000
Epoch 4/6
60000/60000 [==============================] - 506s - loss: 196.6373   
acc_p_c_z:0.01000000
Epoch 5/6
60000/60000 [==============================] - 505s - loss: 194.2171   
lr_nn:0.002500
lr_gmm:0.002500
acc_p_c_z:0.01000000
Epoch 6/6
60000/60000 [==============================] - 494s - loss: 192.4294   


In [0]:
# WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
# Epoch 1/6
# 60000/60000 [==============================] - 101s - loss: 321.7174   
# acc_p_c_z:0.01000000
# Epoch 2/6
# 60000/60000 [==============================] - 373s - loss: 201.2571   
# acc_p_c_z:0.01000000
# Epoch 3/6
# 60000/60000 [==============================] - 422s - loss: 194.2968   
# acc_p_c_z:0.01000000
# Epoch 4/6
# 60000/60000 [==============================] - 430s - loss: 192.9105   
# acc_p_c_z:0.01000000
# Epoch 5/6
# 60000/60000 [==============================] - 427s - loss: 190.1798   
# lr_nn:0.002500
# lr_gmm:0.002500
# acc_p_c_z:0.01000000
# Epoch 6/6
# 60000/60000 [==============================] - 436s - loss: 184.9435   
# <keras.callbacks.History at 0x7fbe77a72e10>

# Result

In [0]:
p_c_z = Lambda(get_gamma, output_shape=(n_centroid,))(z_mean)
sample_output = Model(x, z_mean)
p_c_z_output = Model(x, p_c_z)

accuracy = cluster_acc(np.argmax(p_c_z_output.predict(X,batch_size=batch_size),axis=1), Y)
print('Accuracy: %.2f' % accuracy[0])

Accuracy: 0.01


In [0]:
accuracy[1]

array([[600, 600, 600, ..., 600, 600, 600],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])